In [16]:
# 시간 측정을 위해 추가한 부분_1
import time
import datetime
start=time.time()

# Importing modules
import cv2

from cupy.fft import fft2, fftshift, ifft2, ifftshift
from cupy.fft import fft, ifft
from cupy import tile, real, min, zeros
import cupy as cp

# from numpy.fft import fft2, fftshift, ifft2, ifftshift
# from numpy.fft import fft, ifft
# from numpy import tile, real, min, zeros
import numpy as np

from scipy.signal import firwin
from skimage.filters import gaussian

import math
import time

M_PI = math.pi
eps = 2**(-52)

#확대처리 대상 설정
#videoName = '200Hz_sr2000_0113' #대상 비디오 파일명
# videoName = 'baby_roi' #대상 비디오 파일명
videoName = 'guitar_one_quarter'
videoType = '.avi' #확장자
fourcc = cv2.VideoWriter_fourcc(*'MJPG') #확대과정 후 저장시 mp4로 저장

#Parameter Initializing
Low_Freq =72
High_Freq = 92
alpha = 15
Sampling_Rate = 600
refFrame = 0
NumberOfPyramid = 2
Orientation = 2
sigma = 5
attenuateOtherFreq = 0 # 영역 외 주파수에 대해 베재할 것인가에 대해 True or False
ratioOfFrame = 1

#Video File Importing
fileName = './1_input/{}{}'.format(videoName, videoType)
cap_original = cv2.VideoCapture(fileName)

width = int(cap_original.get(3))
height = int(cap_original.get(4))

size = (width, height)

#Video Information Checking
ret, frame = cap_original.read()
frameX = frame.shape[1]
frameY = frame.shape[0]
nColor = frame.shape[2]
nFrame = int(cap_original.get(cv2.CAP_PROP_FRAME_COUNT))

pyrLayers = 2
rVals = [1]
for i in range(pyrLayers):
    rVals.append(0.5**(i+1))

# 아래 극좌표와 원뿔대 형성 함수를 바탕으로 steerable filter를 형성한다.
def getFilters(dimension, rVals, orientations):
    X = int(dimension[1])
    Y = int(dimension[0])

    defaultTwidth = 1
    twidth = defaultTwidth

    polarGrid = getPolarGrid(dimension)
    count = 0

    angle = cp.array(polarGrid[0]) #픽셀 위치별 각도 값
    rad = cp.array(polarGrid[1]) #픽셀 위치별 거리 값

    mask = getRadialMaskPair(rVals[0], rad, twidth)
    himask = mask[0]
    lomaskPrev = mask[1]

    filters = []
    filters.append(himask)

    for k in range(1, len(rVals)):
        mask = getRadialMaskPair(rVals[k], rad, twidth)
        himask = mask[0]
        lomask = mask[1]

        radMask = himask*lomaskPrev

        for j in range(1, orientations+1):
            angleMask = getAngleMask(j, orientations, angle)
            filters.append(radMask*angleMask/2)

        lomaskPrev = lomask
    filters.append(lomask)

    for k in range(len(filters)):
        filters[k] = cp.array(filters[k])
    return filters

#화면의 너비x높이 크기의 극좌표르를 2차원 배열으로 형성한다.
def getPolarGrid(dimension):
    X = dimension[1]
    Y = dimension[0]
    centerX = int(X / 2)
    centerY = int(Y / 2)

    # Create rectangular grid
    xramp = cp.array([ [(x-int(X/2))/(X/2) for x in range(X)] for y in range(Y)])
    yramp = cp.array([ [(y-int(Y/2))/(Y/2) for x in range(X)] for y in range(Y)])
    angle = cp.arctan2(xramp, yramp)+M_PI/2

    rad = cp.sqrt(xramp**2+yramp**2)
    rad[centerY][centerX] = rad[centerY-1][centerX]

    polarGrid = [angle, rad]
    return polarGrid

#3차원 그래프상으로 보이는 완전 꼬깔 형태에서, 위 아래의 범위를 잘라내고 원뿔대 같은 형태도 만든다.
def getRadialMaskPair(r, rad, twidth): 
    X = int(rad.shape[1])
    Y = int(rad.shape[0])

    log_rad = cp.log2(rad)-cp.log2(r)

    himask = log_rad
    himask[himask>0] = 0
    himask[himask<-twidth] = -twidth
    himask = himask*M_PI/(2*twidth)

    himask = cp.cos(himask)
    lomask = cp.sqrt(1-himask**2)

    mask = [himask, lomask]
    return mask

def getAngleMask(b, orientations, angle):
    order = orientations - 1
    const = (2 ** (2 * order)) * (math.factorial(order) ** 2) / (orientations * math.factorial(2 * order)) # Scaling constant

    angle_ = (M_PI + angle - (M_PI * (b - 1) / orientations)) % (2 * M_PI) - M_PI
    anglemask = 2 * cp.sqrt(const) * (cp.cos(angle_) ** order) * (abs(angle_) < (M_PI / 2))  # Make falloff smooth
    return anglemask

filters = getFilters([frameY, frameX], rVals, Orientation)

def getFilterIDX2(filters, orientations, rVals):
    X = filters[0].shape[1]
    Y = filters[0].shape[0]
    nFilts = len(filters)
    filtIDX = [[None for j in range(orientations)] for i in range(nFilts)]
    croppedFilters = []

    #himask IDX
    filtIDX[0][0] = [y for y in range(Y)]
    filtIDX[0][1] = [x for x in range(X)]
    croppedFilters.append(filters[0])

    #stearable filter IDX
    for k in range(1, nFilts-1, orientations):
        n = int(k/2)+1
        lb_y = int( (Y*(sum(rVals[0:n])-1))/2 )
        ub_y = Y - lb_y
        lb_x = int( (X*(sum(rVals[0:n])-1))/2 )
        ub_x = X - lb_x

        for i in range(orientations):
            filtIDX[k+i][0] = [y + lb_y for y in range(ub_y - lb_y)]
            filtIDX[k+i][1] = [x + lb_x for x in range(ub_x - lb_x)]

        for i in range(orientations):
            croppedFilters.append(filters[k+i][lb_y:ub_y, lb_x:ub_x])


    #lomaskIDX
    lb_y = int( (Y * (sum(rVals) - 1))/2 )
    ub_y = Y - lb_y
    lb_x = int( (X * (sum(rVals) - 1))/2 )
    ub_x = X - lb_x


    filtIDX[nFilts - 1][0] = [y + lb_y for y in range(ub_y - lb_y)]
    filtIDX[nFilts - 1][1] = [x + lb_x for x in range(ub_x - lb_x)]
    croppedFilters.append(filters[nFilts-1][lb_y:ub_y, lb_x:ub_x])

    filterIDX = [croppedFilters, filtIDX]
    return filterIDX


filterIDX = getFilterIDX2(filters, Orientation, rVals)
croppedFilters = filterIDX[0]
filtIDX = filterIDX[1]

def bgr2yiq(img):
    R = img[:, :, 2]
    G = img[:, :, 1]
    B = img[:, :, 0]

    Y = 0.299 * R + 0.587 * G + 0.114 * B
    I = 0.596 * R - 0.274 * G - 0.322 * B
    Q = 0.211 * R - 0.523 * G + 0.312 * B

    img_yiq = cp.zeros([img.shape[0], img.shape[1], img.shape[2]])
    img_yiq[:, :, 0] = cp.array(Y)
    img_yiq[:, :, 1] = cp.array(I)
    img_yiq[:, :, 2] = cp.array(Q)
    #YIQ = np.array([[0.299, 0.587, 0.114],[0.596, -0.274, -0.322],[0.211, -0.523, 0.312]])
    return img_yiq

def yiq2bgr(img):
    Y = img[:, :, 0]
    I = img[:, :, 1]
    Q = img[:, :, 2]

    R = Y+0.956*I+0.621*Q
    G = Y-0.272*I-0.647*Q
    B = Y-1.106*I+1.703*Q

    img_rgb = np.zeros([img.shape[0], img.shape[1], img.shape[2]])
    img_rgb[:, :, 0] = B
    img_rgb[:, :, 1] = G
    img_rgb[:, :, 2] = R
    #YIQ = np.array([[0.299, 0.587, 0.114],[0.596, -0.274, -0.322],[0.211, -0.523, 0.312]])
    return img_rgb


vidFFT = np.zeros([nFrame,frameY,frameX], dtype=np.complex64)

for k in range(0, nFrame):
    clipMat = frame/255
    tVid = bgr2yiq(clipMat)[:,:,0]
    vidFFT[k] = fftshift(fft2(tVid))
    ret, frame = cap_original.read()

cap_original.release()

def fir_window_bp_2(delta, fl, fh):
    length = delta.shape[0]+1
    b = firwin(length, (fl * 2, fh * 2), pass_zero=False)[0:length-1]

    m = delta.shape[1]
    batches = 20
    batch_size = int(m / batches) + 1
    temp = fft(ifftshift(b))
    out = zeros(delta.shape, dtype=delta.dtype)

    i=0
    indexes = (batch_size * i, min((batch_size * (i + 1), m)))
    while (indexes[0]<m):
        indexes = (batch_size * i, min((batch_size * (i + 1), m)))
        freq = fft(delta[:, indexes[0]:indexes[1]], axis=0) * tile(temp, (
            delta.shape[2], indexes[1] - indexes[0], 1)).swapaxes(0, 2)
        out[:, indexes[0]:indexes[1]] = real(ifft(freq, axis=0))
        i += 1
        indexes = (batch_size * i, min((batch_size * (i + 1), m)))
    return out

def amplitude_weighted_blur(x, weight, sigma):
    if sigma != 0:
        return gaussian(x*weight, sigma, mode="wrap") / gaussian(weight, sigma, mode="wrap")
    return x

magnifiedLumaFFT = np.zeros([nFrame, frameY, frameX], dtype=np.complex64)
numLevels = len(filters)

for level in range(1, numLevels-1):
    X = len(croppedFilters[level][0])
    Y = len(croppedFilters[level])
    lb_x = filtIDX[level][1][0]
    ub_x = filtIDX[level][1][-1]+1
    lb_y = filtIDX[level][0][0]
    ub_y = filtIDX[level][0][-1]+1

    # 1. 기준프레임 설정
    clipMat = croppedFilters[level] * vidFFT[refFrame][lb_y:ub_y, lb_x:ub_x]
    pyrRef = ifft2(ifftshift(clipMat))
    pyrRefPhaseOrig = pyrRef / abs(pyrRef)
    pyrRef = np.angle(pyrRef)

    delta = np.zeros([nFrame,Y,X], dtype=np.float16)
    matCheck = []
    
    # 2. 각 프레임간 차이 계산
    for frameIDX in range(0, nFrame):
        filterResponse = ifft2(ifftshift( croppedFilters[level] * vidFFT[frameIDX][lb_y:ub_y, lb_x:ub_x] ))
        pyrCurrent = np.angle(filterResponse)
        clipMat1 = pyrCurrent - pyrRef
        clipMat2 = M_PI + clipMat1
        clipMat3 = clipMat2%(2*M_PI)
        clipMat4 = clipMat3 - M_PI
        clipMat = clipMat4
        delta[frameIDX] = clipMat

    # 3. 픽셀 변화 양상에 대한 band pass filtering
    delta_1 = fir_window_bp_2(delta, Low_Freq / Sampling_Rate, High_Freq / Sampling_Rate)  # Finite Impulse Response filter

    for frameIDX in range(0, nFrame):
        Phase = delta_1[frameIDX]

        originalLevel = ifft2(ifftshift(croppedFilters[level]*vidFFT[frameIDX][lb_y:ub_y, lb_x:ub_x]))

        if (sigma != 0):
            Phase = amplitude_weighted_blur(Phase, abs(originalLevel)+eps, sigma)
        
        # 4. alpha 계수 곱 및 병합
        Phase = alpha*Phase

        if (attenuateOtherFreq):
            tempOrig = abs(originalLevel)*pyrRefPhaseOrig
        else:
            tempOrig = originalLevel

        tempTransformOut = np.exp(1j*Phase)*tempOrig


        A = croppedFilters[level]
        B = fftshift(fft2(tempTransformOut))
        curLevelFrame = 2 * A * B

        matClip = magnifiedLumaFFT[frameIDX][lb_y:ub_y, lb_x:ub_x]
        magnifiedLumaFFT[frameIDX][lb_y:ub_y, lb_x:ub_x] = matClip + curLevelFrame

#5. lowpass residual 처리
level = len(filters)-1
lb_x = filtIDX[level][1][0]
ub_x = filtIDX[level][1][-1] + 1
lb_y = filtIDX[level][0][0]
ub_y = filtIDX[level][0][-1] + 1
for frameIDX in range(0, nFrame):
    lowpassFrame = vidFFT[frameIDX][lb_y:ub_y, lb_x:ub_x]*(croppedFilters[level]**2)
    matClip = magnifiedLumaFFT[frameIDX][lb_y:ub_y, lb_x:ub_x] + lowpassFrame
    magnifiedLumaFFT[frameIDX][lb_y:ub_y, lb_x:ub_x] = matClip

    
##########################Saving
cap_original = cv2.VideoCapture(fileName)

fourcc = cv2.VideoWriter_fourcc(*'MJPG')
obj_out = cv2.VideoWriter('./output/(color) '+ str(Low_Freq) + '~' + str(High_Freq) + ', a=' + str(alpha) + ' {}.avi'.format(videoName)
                          , fourcc, 30.0, (frameX, frameY))

ret, frame = cap_original.read()

for k in range(0, nFrame):
    clipMat1 = magnifiedLumaFFT[k]
    clipMat = ifft2(ifftshift(clipMat1))
    magnifiedLuma = np.real(clipMat)
    #명암에 대한 데이터만 적용하기 위해 yiq변환해서 Y 값만 가져온다.
    clipMat2 = bgr2yiq(frame)
    clipMat3 = magnifiedLuma*255
    clipMat2[:,:,0] = clipMat3
    
    #그 외의 Q, I는 그대로 사용할 것이므로 변환된 Y만 원본 영상에 대입한 뒤 다시 RGB로 변환한다.
    outFrame = yiq2bgr(clipMat2)
    
    outFrame[outFrame > 255] = 255
    outFrame[outFrame < 0] = 0
    outFrame = np.uint8(outFrame)
    obj_out.write(outFrame)
    ret, frame = cap_original.read()

cap_original.release()
obj_out.release()

sec=time.time()-start
times=str(datetime.timedelta(seconds=sec)).split(".")
times=times[0]
print("실행시간 :" + times)
print(size)


실행시간 :0:00:08
(216, 96)


In [1]:
# 필요한 라이브러리 및 모듈
import numpy as np
import cv2
from scipy.signal import butter, lfilter
import time

# Parameter Initializing
start = time.time()
# Read Video file
videoName = 'cantilever_oneOversixteen'
filename = "./1_input/{}.avi".format(videoName)
cap = cv2.VideoCapture(filename)
video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

numOfPyramids = 3 # Laplacian, Gaussian Pyramid Level

SR = 2200 # SR : Sampling Rate
Fn = SR / 2
T = 1 / SR  # T : Period

# Frequency unit : Hz
Low_freq = 350
High_freq = 370

# alpha is Magnification coefficient
alpha = 15

# RGB to Gray Scale

# 첫번째 프레임 읽기
ret, frame = cap.read()

# 영상의 사이즈 확인
frame_size = frame.shape
print("Shape of image is : ", frame.shape)

# Gray Sacle 이미지로 변환
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
print("Shape of image is : ", gray.shape)


# 이미지 명암값 대비를 크게 하여 더욱 선명하게 함
def CLAHE(input):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    output = clahe.apply(input)
    return output


# pyramid 안쓰고, resize함수로 피라미드 정의
def pyramidDown(input):
    blur = cv2.GaussianBlur(input, (5, 5), 0)
    output = cv2.resize(blur, None, fx=0.5, fy=0.5)
    return output


def pyramidUp(input):
    input = cv2.resize(input, None, fx=2, fy=2, interpolation=cv2.INTER_LINEAR)
    output = cv2.GaussianBlur(input, (5, 5), 0)
    return output

def bgr2yiq(img):
    R = img[:, :, 2]; G = img[:, :, 1]; B = img[:, :, 0]

    Y = 0.299 * R + 0.587 * G + 0.114 * B
    I = 0.596 * R - 0.274 * G - 0.322 * B
    Q = 0.211 * R - 0.523 * G + 0.312 * B

    img_yiq = np.zeros([img.shape[0], img.shape[1], img.shape[2]])
    img_yiq[:, :, 0] = Y; img_yiq[:, :, 1] = I; img_yiq[:, :, 2] = Q
    return img_yiq

def yiq2bgr(img):
    Y = img[:, :, 0]; I = img[:, :, 1]; Q = img[:, :, 2]

    R = Y + 0.956 * I + 0.621 * Q
    G = Y - 0.272 * I - 0.647 * Q
    B = Y - 1.106 * I + 1.703 * Q

    img_rgb = np.zeros([img.shape[0], img.shape[1], img.shape[2]])
    img_rgb[:, :, 0] = B; img_rgb[:, :, 1] = G; img_rgb[:, :, 2] = R
    return img_rgb

# Generate Pyramid
GauSeq = [[] for n in range(0,numOfPyramids)]
LapSeq = [[] for n in range(0,numOfPyramids)]

colorData = []; k=0
while ret:
    colorData.append(bgr2yiq(frame))
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = CLAHE(gray)

    # Normalize intensity
    gray = gray.astype('float64')
    gray = gray / 255

    Gau = [gray]
    Down = pyramidDown(gray)
    Gau.append(Down)

    for i in range(0, numOfPyramids-1):
        Down = pyramidDown(Down)
        Gau.append(Down)

    GauCopy = []
    GauCopy[:] = Gau
    GauCopy.reverse()

    for i in range(0, numOfPyramids):
        GauSeq[i].append(GauCopy.pop())

    Lap = []
    for i in range(0, len(LapSeq)):
        Up = pyramidUp(Gau.pop())
        Lap.append(Gau[len(Gau) - 1] - Up)

    for i in range(0, len(LapSeq)):
        LapSeq[i].append(Lap.pop())
    ret, frame = cap.read(); k+=1


#### filtering function
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band', analog=False)
    y=lfilter(b, a, data, axis=0)
    return y


def filtering(target, lowFrequency, highFrequency, samplingRate, order=5):
    timeStack = butter_bandpass_filter(target, lowFrequency, highFrequency, samplingRate, order)
    for k in range(0, len(target)):
        target[k] += alpha * timeStack[k]
    return target


# Bandpass filtering
for i in range(0, numOfPyramids):
    LapSeq[i] = filtering(LapSeq[i], Low_freq, High_freq, SR)

# Gaussian + Laplacian ->
Rec = [[] for n in range(0,numOfPyramids)]

for i in range(0, len(LapSeq[0])):
    Rec[len(Rec)-1].append(LapSeq[len(Rec)-1][i] + GauSeq[len(Rec)-1][i])
    Rec[len(Rec)-1][i] = pyramidUp(Rec[len(Rec)-1][i])

for Lv in range(1, len(Rec)):
    Lv_inv = len(Rec) - 1 - Lv
    for i in range(0, len(LapSeq[0])):
        Rec[Lv_inv].append(LapSeq[Lv_inv][i] + Rec[Lv_inv+1][i])
        if Lv_inv != 0:
            Rec[Lv_inv][i] = pyramidUp(Rec[Lv_inv][i])
for i in range(0, len(Rec[0])):
    Rec[0][i][Rec[0][i] < 0] = 0
    Rec[0][i][Rec[0][i] > 1] = 1
    Rec[0][i] *= 255

#
#
# Rec2 = []
# for i in range(0, len(LapSeq[0])):
#     Sum = LapSeq[2][i] + GauSeq[2][i]
#     Rec2.append(Sum)
#
# for i in range(0, len(LapSeq[0])):
#     Rec2[i] = pyramidUp(Rec2[i])
#
# Rec1 = []
# for i in range(0, len(LapSeq[0])):
#     Sum = LapSeq[1][i] + Rec2[i]
#     Rec1.append(Sum)
#
# for i in range(0, len(LapSeq[0])):
#     Rec1[i] = pyramidUp(Rec1[i])
#
# Rec0 = []
# for i in range(0, len(LapSeq[0])):
#     Sum = LapSeq[0][i] + Rec1[i]
#     Rec0.append(Sum)
#
#
# # 형변환 및 오버플로우 방지
# for i in range(0, len(Rec0)):
#     Rec0[i][Rec0[i] < 0] = 0
#     Rec0[i][Rec0[i] > 1] = 1
#     Rec0[i] *= 255
#     Rec0[i] = Rec0[i].astype('uint8')


# Output 파일 생성

size = Rec[0][0].shape

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('./output/' + 'alpha_' + str(alpha) + 'output.mp4', fourcc, 30.0, (size[1], size[0]))

def rgbNormalize(img):
    for i in range(0,3):
        ub = max(img[:,:,i])

for i in range(0, len(Rec[0])):
    colorData[i][:,:,0] = Rec[0][i]
    clipImg = yiq2bgr(colorData[i])
    clipImg[clipImg < 0] = 0; clipImg[clipImg > 255] = 255
    out.write(clipImg.astype('uint8'))

out.release()

print("whole process time costs: {}sec".format(time.time() - start))





Shape of image is :  (64, 160, 3)
Shape of image is :  (64, 160)
whole process time costs: 1.0880622863769531sec
